프로세스 마이닝 입력 데이터 구조와 프로세스 분석 기본 개념

-프로세스 분석 기본 기념 예제

In [18]:
from dateutil.parser import parse
# import datetime

# a = '2017-06-10 10:00'
# datetime.datetime.strptime(a, '%Y-%m-%d %H:%M')

datetime.datetime(2017, 6, 10, 10, 0)

In [72]:
caseId = ['400909207-1', '400909207-2', '400909207-3', '400909207-4']
purchaseOrder = ['2017-06-10 10:00','2017-06-10 10:00','2017-06-10 10:00','2017-06-10 10:00']
paymentChange = ['2017-06-11 10:00','2017-06-12 10:00','2017-06-13 10:00','2017-06-13 14:00']
warehousing = ['2017-06-20 14:00','2017-06-27 14:00','2017-06-30 14:00','2017-06-28 16:00']
invoiceReceipt = ['2017-06-21 14:00','2017-06-28 14:00','2017-07-01 14:00','2017-07-08 14:00']


ex1 = pd.DataFrame(data = {'caseId' : caseId,'구매오더생성' : purchaseOrder, '계약납기일변경' : paymentChange,\
                           '자재입고' : warehousing,'송장수령': invoiceReceipt})



def duration(start, finish):
    sta_t = parse(start)
    fin_t = parse(finish)
    duration_sec = fin_t - sta_t
    duration_hour = duration_sec.total_seconds()//3600
    return duration_hour

def duration_list(start_l, finish_l):
    answer = []
    for sta, fin in zip(start_l, finish_l):
        answer.append(duration(sta, fin))
        
    return answer
    

def get_pvt():
    
    dataset = {
        '구매오더생성>계약납기일변경' : duration_list(purchaseOrder, paymentChange) , 
        '계약납기일변경>자재입고': duration_list(paymentChange, warehousing) , 
        '자재입고>송장수령': duration_list(warehousing, invoiceReceipt),
    }
    
    pv = pd.DataFrame(data = dataset, index=caseId)
    pv['케이스수행기간'] = pv.apply(sum, axis=1)
    return pv
    
pv1 = get_pvt()

In [80]:
pv1

,계약납기일변경>자재입고,구매오더생성>계약납기일변경,자재입고>송장수령,케이스수행기간
400909207-1,220.0,24.0,24.0,268.0
400909207-2,364.0,48.0,24.0,436.0
400909207-3,412.0,72.0,24.0,508.0
400909207-4,362.0,76.0,238.0,676.0


In [105]:
def get_indice(col):
    answer = [col.sum(), col.mean(), col.median(), col.max(), col.min()]
    answer = [round(x) for x in answer]
    return answer


def get_indice_table(pv):
    indice = {}
    
    for col in pv.columns:
        indice[col] = get_indice(pv[col])
    
    idx_list = ['전체','평균','중간값','최댓값','최솟값']
    
    indice_table = pd.DataFrame(data = indice, index = idx_list)
    
    return indice_table

get_indice_table(pv1)

,계약납기일변경>자재입고,구매오더생성>계약납기일변경,자재입고>송장수령,케이스수행기간
전체,1358.0,220.0,310.0,1888.0
평균,340.0,55.0,78.0,472.0
중간값,363.0,60.0,24.0,472.0
최댓값,412.0,76.0,238.0,676.0
최솟값,220.0,24.0,24.0,268.0
